*Table of Contents*
1. 필요모듈 import & file read
2. Top 200 author 뽑기
3. SVD 적용하기
---

# 필요 모듈 Import & Data read

In [3]:
# 모듈 import
from collections import Counter
from datetime import timedelta, datetime
import re

In [5]:
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import hangul_font

Hangul font is set!


In [6]:
from sklearn.decomposition import TruncatedSVD

##### Data Read

In [2]:
magazine = pd.read_csv('magazine_final.csv')

In [7]:
metadata = pd.read_csv('metadata_final.csv')

In [8]:
users = pd.read_csv('users_final.csv')

In [9]:
read = pd.read_csv('read_final.csv')

# Top200 작가 선정
- Top200 작가 선정 (가장 글이 많이 읽힌 200명의 작가)
    - reader 기반 작가 추천: top200 작가를 대상으로 진행
    - 머신러닝으로 추천을 하기 위해 데이터 축소
- 독자별 읽은 글의 수/ 작가별 작성한 글의 수 구하기
- 15000명의 독자/ 200명의 작가로 구성된 pivot 생성
    - 자신이 읽은 글 중 해당 작가의 글의 비중과 작가가 쓴 전체 글 중 해당 독자가 읽은 글의 비중을 가중치를 두고 계산하여 value로 사용

## 글 많이 읽힌 작가 top 200 뽑기
- Read + Metadata Files 

In [14]:
read_final = pd.merge(left = read, right=metadata, left_on="article_id", right_on="article_id", how="left")

In [15]:
read_final.drop(columns=["display_url", "keyword_list", "magazine_id", "reg_ts", "sub_title", "title"], inplace=True)

In [18]:
read_top_200_count = read_final.groupby("author_id")["readers_id"].agg("count").sort_values(ascending=False).reset_index()

In [19]:
read_top_200_count.head()

,author_id,readers_id
0,@brunch,402451
1,@tenbody,319864
2,@jordan777,311157
3,@dailylife,256239
4,@binkond,156974


In [20]:
# 상위 200명만 저장
read_top_200_count = read_top_200_count.iloc[:200, :]

In [21]:
read_top_200_count.author_id.nunique()

200

In [22]:
# 컬럼명 변경
read_top_200_count.rename(columns={"readers_id":"reader_id_count"}, inplace=True)
read_top_200_count

,author_id,reader_id_count
0,@brunch,402451
1,@tenbody,319864
2,@jordan777,311157
3,@dailylife,256239
4,@binkond,156974
...,...,...
195,@ssuujin,16255
196,@finance1026,16251
197,@alicemelbourne,16248
198,@sustainability,16169


In [100]:
# read_top_200_count(reader_id_count) csv파일로 저장
read_top_200_count.to_csv("Read_Author_Top_200_Count.csv", index=False)

#### 최종 Read_Top_200 DataFrame

In [23]:
read_top_200 = pd.merge(left=read_final, right=read_top_200_count, left_on="author_id", right_on="author_id", how="right")

In [24]:
read_top_200.author_id.nunique()

200

In [103]:
# top200 csv 파일로저장
read_top_200.to_csv("Read_Author_Top_200.csv", index=False)

In [57]:
read_top_200.head()

,dt,hr,readers_id,article_id,author_id,article_number,reader_id_count
0,20181001,0,#0a3d493f3b2318be80f391eaa00bfd1c,@readme999_140,@readme999,140.0,31548
1,20181001,0,#88f2c6beaa352f808019befebd9f8bd0,@readme999_145,@readme999,145.0,31548
2,20181001,0,#b1e45aeff4915ce4e2ab350546bd6689,@readme999_165,@readme999,165.0,31548
3,20181001,0,#0e97d1dc7ee6bf7d6fa45eb5d3133343,@readme999_161,@readme999,161.0,31548
4,20181001,0,#0e97d1dc7ee6bf7d6fa45eb5d3133343,@readme999_161,@readme999,161.0,31548


## 작가의 글 수 합계, 독자의 읽은 수 합계 구하기

In [26]:
# 작가별 작성한 글의 총 수
df_author = pd.DataFrame(read_top_200.groupby('author_id').count()['article_number']).T

In [27]:
# 독자별 읽은 글의 총 수
df_reader = pd.DataFrame(read_top_200.groupby('readers_id').count()['article_number'])

In [28]:
# 독자별 읽은 작가의 수
df_reader_author = pd.DataFrame(read_top_200.groupby('readers_id').nunique()['author_id'])

In [38]:
# 독자별로 읽은 작가의 수가 많은 순서대로 상위 15000명 sorting
readers_id_df=df_reader_author.sort_values('author_id', ascending=False)[:15000].reset_index()
readers_id_df

,readers_id,author_id
0,#6f55b5508f0c31ae3456621c23c1f6a9,194
1,#71ca8074251ffb285a1ee286fde639b8,193
2,#9ef170cacaa30f62d6de34a5ad42a37e,193
3,#dcbfec386ba5af556b70bb98787c35ea,188
4,#64ed07be4247d4f03737119faad50903,185
...,...,...
14995,#446d2433642d440b48a45509a8ee37a5,24
14996,#4193ff1bcca6a3372f9faae2339d5f7a,24
14997,#db4e97d6b0ba8442cce7a3c89d08c58e,24
14998,#134b1867c4d70127b97d2ca5627d1293,24


In [39]:
readers_id_ls = user_id_df.drop('author_id', axis=1)
readers_id_ls

,readers_id
0,#6f55b5508f0c31ae3456621c23c1f6a9
1,#71ca8074251ffb285a1ee286fde639b8
2,#9ef170cacaa30f62d6de34a5ad42a37e
3,#dcbfec386ba5af556b70bb98787c35ea
4,#64ed07be4247d4f03737119faad50903
...,...
14995,#446d2433642d440b48a45509a8ee37a5
14996,#4193ff1bcca6a3372f9faae2339d5f7a
14997,#db4e97d6b0ba8442cce7a3c89d08c58e
14998,#134b1867c4d70127b97d2ca5627d1293


In [112]:
df_author.columns

Index(['@01038273527', '@13july', '@aemae-human', '@ahronjeon', '@alexkang',
       '@alicemelbourne', '@allstay', '@am327', '@anetmom', '@angiesongc9sx',
       ...
       '@workerhanee', '@worknlife', '@ws820512', '@x-xv', '@yemaya', '@yemyo',
       '@yeonboon', '@yoonjikwon', '@yoooong', '@yumileewyky'],
      dtype='object', name='author_id', length=200)

In [91]:
# read_top_200.drop(['dt', 'hr', 'article_id'], axis=1, inplace=True)

In [30]:
read_top_200.columns

Index(['dt', 'hr', 'readers_id', 'article_id', 'author_id', 'article_number',
       'reader_id_count'],
      dtype='object')

## 작가 * 독자 pivot 만들기

In [31]:
df_pivot = read_top_200.pivot_table(values='article_number', index=['readers_id'], columns='author_id', aggfunc='count')

In [32]:
df2 = pd.concat([df_pivot, df_author], axis=0)

In [33]:
df2

author_id,@01038273527,@13july,@aemae-human,@ahronjeon,@alexkang,@alicemelbourne,@allstay,@am327,@anetmom,@angiesongc9sx,...,@workerhanee,@worknlife,@ws820512,@x-xv,@yemaya,@yemyo,@yeonboon,@yoonjikwon,@yoooong,@yumileewyky
#00001ba6ca8d87d2fc34d626ba9cfe6f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#0000e87158c1426d6ffb72cebac6cb64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#0000fdba8f35c76eacab74c5c6bc7f1a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#000127ad0f1981cae1292efdb228f0e9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#0001485b31e8f02c1ce117ceb4f41560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#fffe67ecc0056dd26ae00511957c5a2b,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#ffff69451ff594425637015500410a13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#ffff8d99b9caef8ad1b95cecf0b8eef4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#ffffc97f29a10678203330ec0b6bf138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df3 = pd.concat([df2, df_reader], axis=1)

In [35]:
df3.reset_index(inplace=True)
df3.rename(columns={'index':'readers_id'}, inplace=True)
df3

,readers_id,@01038273527,@13july,@aemae-human,@ahronjeon,@alexkang,@alicemelbourne,@allstay,@am327,@anetmom,...,@worknlife,@ws820512,@x-xv,@yemaya,@yemyo,@yeonboon,@yoonjikwon,@yoooong,@yumileewyky,article_number
0,#00001ba6ca8d87d2fc34d626ba9cfe6f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1,#0000e87158c1426d6ffb72cebac6cb64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,#0000fdba8f35c76eacab74c5c6bc7f1a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,#000127ad0f1981cae1292efdb228f0e9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0
4,#0001485b31e8f02c1ce117ceb4f41560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239015,#fffe67ecc0056dd26ae00511957c5a2b,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
239016,#ffff69451ff594425637015500410a13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
239017,#ffff8d99b9caef8ad1b95cecf0b8eef4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
239018,#ffffc97f29a10678203330ec0b6bf138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [40]:
# pivot table 만들기
pivot_15000 = pd.merge(left=readers_id_ls, right=df3, left_on='readers_id', right_on='readers_id', how='left')

In [41]:
pivot_15000_fill = pivot_15000.fillna(0)

In [42]:
pivot_15000_fill = pd.concat([pivot_15000_fill, df_author], axis=0)

In [43]:
pivot_15000_fill.set_index('readers_id', drop=True, inplace=True)
pivot_15000_fill

,@01038273527,@13july,@aemae-human,@ahronjeon,@alexkang,@alicemelbourne,@allstay,@am327,@anetmom,@angiesongc9sx,...,@worknlife,@ws820512,@x-xv,@yemaya,@yemyo,@yeonboon,@yoonjikwon,@yoooong,@yumileewyky,article_number
readers_id,,,,,,,,,,,,,,,,,,,,,
#6f55b5508f0c31ae3456621c23c1f6a9,12.0,41.0,7.0,2.0,61.0,3.0,56.0,36.0,17.0,55.0,...,40.0,36.0,4.0,56.0,60.0,18.0,0.0,10.0,5.0,5367.0
#71ca8074251ffb285a1ee286fde639b8,11.0,16.0,13.0,22.0,3.0,4.0,84.0,9.0,24.0,5.0,...,2.0,60.0,29.0,13.0,37.0,40.0,1.0,18.0,29.0,5845.0
#9ef170cacaa30f62d6de34a5ad42a37e,6.0,35.0,19.0,0.0,23.0,0.0,38.0,41.0,38.0,25.0,...,44.0,15.0,0.0,50.0,20.0,20.0,2.0,30.0,0.0,5380.0
#dcbfec386ba5af556b70bb98787c35ea,17.0,4.0,35.0,2.0,18.0,1.0,16.0,19.0,14.0,5.0,...,12.0,11.0,3.0,13.0,13.0,12.0,1.0,3.0,4.0,2056.0
#64ed07be4247d4f03737119faad50903,26.0,50.0,0.0,10.0,41.0,0.0,74.0,35.0,13.0,15.0,...,59.0,24.0,4.0,31.0,61.0,19.0,0.0,17.0,5.0,5012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#4193ff1bcca6a3372f9faae2339d5f7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0
#db4e97d6b0ba8442cce7a3c89d08c58e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0
#134b1867c4d70127b97d2ca5627d1293,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58.0


In [44]:
for i in range(15001):
    for j in range(201):
        pivot_15000_fill.iloc[i,j] = (pivot_15000_fill.iloc[i,j] / pivot_15000_fill.iloc[i, 200]) * 0.5 + (pivot_15000_fill.iloc[i,j]/ pivot_15000_fill.iloc[15000, j]) * 0.5 

pivot_15000_fill

,@01038273527,@13july,@aemae-human,@ahronjeon,@alexkang,@alicemelbourne,@allstay,@am327,@anetmom,@angiesongc9sx,...,@worknlife,@ws820512,@x-xv,@yemaya,@yemyo,@yeonboon,@yoonjikwon,@yoooong,@yumileewyky,article_number
readers_id,,,,,,,,,,,,,,,,,,,,,
#6f55b5508f0c31ae3456621c23c1f6a9,0.001309,0.004895,0.000790,0.000247,0.006674,0.000372,0.005868,0.003967,0.001726,0.006514,...,0.004675,0.004003,0.000412,0.006365,0.006691,0.001914,0.000000,0.001168,0.000548,NaN
#71ca8074251ffb285a1ee286fde639b8,0.001116,0.001788,0.001368,0.002549,0.000305,0.000465,0.008162,0.000923,0.002254,0.000554,...,0.000219,0.006214,0.002766,0.001378,0.003844,0.003950,0.000104,0.001965,0.002960,NaN
#9ef170cacaa30f62d6de34a5ad42a37e,0.000653,0.004171,0.002140,0.000000,0.002511,0.000000,0.003973,0.004508,0.003850,0.002955,...,0.005133,0.001664,0.000000,0.005671,0.002226,0.002123,0.000223,0.003497,0.000000,NaN
#dcbfec386ba5af556b70bb98787c35ea,0.004405,0.001078,0.009200,0.000547,0.004670,0.000274,0.004077,0.004944,0.003522,0.001342,...,0.003203,0.002873,0.000759,0.003428,0.003400,0.003077,0.000262,0.000800,0.001039,NaN
#64ed07be4247d4f03737119faad50903,0.003008,0.006299,0.000000,0.001301,0.004757,0.000000,0.008243,0.004088,0.001406,0.001876,...,0.007285,0.002827,0.000438,0.003728,0.007205,0.002146,0.000000,0.002098,0.000581,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#4193ff1bcca6a3372f9faae2339d5f7a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
#db4e97d6b0ba8442cce7a3c89d08c58e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037054,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
#134b1867c4d70127b97d2ca5627d1293,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008632,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [45]:
pivot_15000_fill.drop(['article_number'], axis=1, inplace=True)
pivot_15000_fill

,@01038273527,@13july,@aemae-human,@ahronjeon,@alexkang,@alicemelbourne,@allstay,@am327,@anetmom,@angiesongc9sx,...,@workerhanee,@worknlife,@ws820512,@x-xv,@yemaya,@yemyo,@yeonboon,@yoonjikwon,@yoooong,@yumileewyky
readers_id,,,,,,,,,,,,,,,,,,,,,
#6f55b5508f0c31ae3456621c23c1f6a9,0.001309,0.004895,0.000790,0.000247,0.006674,0.000372,0.005868,0.003967,0.001726,0.006514,...,0.001105,0.004675,0.004003,0.000412,0.006365,0.006691,0.001914,0.000000,0.001168,0.000548
#71ca8074251ffb285a1ee286fde639b8,0.001116,0.001788,0.001368,0.002549,0.000305,0.000465,0.008162,0.000923,0.002254,0.000554,...,0.010773,0.000219,0.006214,0.002766,0.001378,0.003844,0.003950,0.000104,0.001965,0.002960
#9ef170cacaa30f62d6de34a5ad42a37e,0.000653,0.004171,0.002140,0.000000,0.002511,0.000000,0.003973,0.004508,0.003850,0.002955,...,0.008823,0.005133,0.001664,0.000000,0.005671,0.002226,0.002123,0.000223,0.003497,0.000000
#dcbfec386ba5af556b70bb98787c35ea,0.004405,0.001078,0.009200,0.000547,0.004670,0.000274,0.004077,0.004944,0.003522,0.001342,...,0.001754,0.003203,0.002873,0.000759,0.003428,0.003400,0.003077,0.000262,0.000800,0.001039
#64ed07be4247d4f03737119faad50903,0.003008,0.006299,0.000000,0.001301,0.004757,0.000000,0.008243,0.004088,0.001406,0.001876,...,0.001392,0.007285,0.002827,0.000438,0.003728,0.007205,0.002146,0.000000,0.002098,0.000581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#4193ff1bcca6a3372f9faae2339d5f7a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
#db4e97d6b0ba8442cce7a3c89d08c58e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037054,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
#134b1867c4d70127b97d2ca5627d1293,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008632,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [47]:
pivot_15000_fill = pivot_15000_fill.drop(pivot_15000_fill.iloc[-1, :])

In [48]:
# csv파일로 저장
pivot_15000_fill.to_csv('pivot_15000_fina.csv', encoding='utf-8')

In [49]:
pivot_15000_fill

,@01038273527,@13july,@aemae-human,@ahronjeon,@alexkang,@alicemelbourne,@allstay,@am327,@anetmom,@angiesongc9sx,...,@workerhanee,@worknlife,@ws820512,@x-xv,@yemaya,@yemyo,@yeonboon,@yoonjikwon,@yoooong,@yumileewyky
readers_id,,,,,,,,,,,,,,,,,,,,,
#6f55b5508f0c31ae3456621c23c1f6a9,0.001309,0.004895,0.000790,0.000247,0.006674,0.000372,0.005868,0.003967,0.001726,0.006514,...,0.001105,0.004675,0.004003,0.000412,0.006365,0.006691,0.001914,0.000000,0.001168,0.000548
#71ca8074251ffb285a1ee286fde639b8,0.001116,0.001788,0.001368,0.002549,0.000305,0.000465,0.008162,0.000923,0.002254,0.000554,...,0.010773,0.000219,0.006214,0.002766,0.001378,0.003844,0.003950,0.000104,0.001965,0.002960
#9ef170cacaa30f62d6de34a5ad42a37e,0.000653,0.004171,0.002140,0.000000,0.002511,0.000000,0.003973,0.004508,0.003850,0.002955,...,0.008823,0.005133,0.001664,0.000000,0.005671,0.002226,0.002123,0.000223,0.003497,0.000000
#dcbfec386ba5af556b70bb98787c35ea,0.004405,0.001078,0.009200,0.000547,0.004670,0.000274,0.004077,0.004944,0.003522,0.001342,...,0.001754,0.003203,0.002873,0.000759,0.003428,0.003400,0.003077,0.000262,0.000800,0.001039
#64ed07be4247d4f03737119faad50903,0.003008,0.006299,0.000000,0.001301,0.004757,0.000000,0.008243,0.004088,0.001406,0.001876,...,0.001392,0.007285,0.002827,0.000438,0.003728,0.007205,0.002146,0.000000,0.002098,0.000581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
#446d2433642d440b48a45509a8ee37a5,0.000000,0.005708,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
#4193ff1bcca6a3372f9faae2339d5f7a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
#db4e97d6b0ba8442cce7a3c89d08c58e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037054,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# SVD 적용하기

In [50]:
pivot_t = pivot_15000_fill.values.T

In [51]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(pivot_t)
matrix.shape

(200, 12)

In [52]:
corr = np.corrcoef(matrix)
corr.shape

(200, 200)

In [53]:
list(df_author.columns)

['@01038273527',
 '@13july',
 '@aemae-human',
 '@ahronjeon',
 '@alexkang',
 '@alicemelbourne',
 '@allstay',
 '@am327',
 '@anetmom',
 '@angiesongc9sx',
 '@ansyd',
 '@anti-essay',
 '@artinsight',
 '@bang1999',
 '@binkond',
 '@bjh4372',
 '@blue2046',
 '@boboc',
 '@bonfire',
 '@bookerbuker',
 '@bookfit',
 '@bookguru',
 '@boot0715',
 '@brunch',
 '@brunch1uhl',
 '@bzup',
 '@cathongzo',
 '@cathykimmd',
 '@chofang1',
 '@chojeremy',
 '@cli-annah',
 '@comeintothe',
 '@conbus',
 '@contigo',
 '@coolivaworld',
 '@cosmos-j',
 '@dahong',
 '@dahyun0421',
 '@dailylife',
 '@dalda',
 '@daljasee',
 '@dancingsnail',
 '@daro',
 '@ddamimovie',
 '@deckey1985',
 '@dizzo',
 '@dong02',
 '@dosa1000',
 '@doyeonsunim',
 '@dprnrn234',
 '@dryjshin',
 '@eastgo',
 '@ehahdp83',
 '@elara1020',
 '@ellieyang47uu',
 '@englishspeaking',
 '@eundang',
 '@expediakr',
 '@finance1026',
 '@flyjy724',
 '@forchoon',
 '@futurewave',
 '@glamjulie',
 '@gorrajeju',
 '@heaven',
 '@hitchwill',
 '@hjl0520',
 '@holidaymemories',
 '@honeytip

In [54]:
pivot_t

array([[0.00130935, 0.00111643, 0.00065332, ..., 0.        , 0.        ,
        0.        ],
       [0.00489485, 0.00178829, 0.00417065, ..., 0.        , 0.        ,
        0.        ],
       [0.00078989, 0.00136789, 0.0021397 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.0001039 , 0.00022258, ..., 0.        , 0.        ,
        0.        ],
       [0.00116801, 0.00196529, 0.00349729, ..., 0.        , 0.        ,
        0.        ],
       [0.00054847, 0.00296017, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [55]:
author = pivot_15000_fill.columns
author_list = list(df_author.columns)
coffey_hands = author_list.index("@roysday")

In [56]:
corr_coffey_hands  = corr[coffey_hands]
list(author[(corr_coffey_hands >= 0.9)])[:50]

['@forchoon',
 '@jinbread',
 '@kakao-it',
 '@mobiinside',
 '@plusx',
 '@puzzle87',
 '@roysday',
 '@thinkaboutlove',
 '@windydog']